In [1]:
import pandas as pd
import tensorflow as tf
import numpy as np

In [2]:
TRAIN_DATA_URL = 'https://storage.googleapis.com/tf-datasets/titanic/train.csv'
TEST_DATA_URL = 'https://storage.googleapis.com/tf-datasets/titanic/eval.csv'

train_file_path = tf.keras.utils.get_file('train.csv', TRAIN_DATA_URL)
test_file_path = tf.keras.utils.get_file('test.csv', TEST_DATA_URL)

In [3]:
train_file_path, test_file_path

('/home/kdd/.keras/datasets/train.csv', '/home/kdd/.keras/datasets/test.csv')

In [4]:
np.set_printoptions(precision=3, suppress=True)

In [5]:
!head {train_file_path}

survived,sex,age,n_siblings_spouses,parch,fare,class,deck,embark_town,alone
0,male,22.0,1,0,7.25,Third,unknown,Southampton,n
1,female,38.0,1,0,71.2833,First,C,Cherbourg,n
1,female,26.0,0,0,7.925,Third,unknown,Southampton,y
1,female,35.0,1,0,53.1,First,C,Southampton,n
0,male,28.0,0,0,8.4583,Third,unknown,Queenstown,y
0,male,2.0,3,1,21.075,Third,unknown,Southampton,n
1,female,27.0,0,2,11.1333,Third,unknown,Southampton,n
1,female,14.0,1,0,30.0708,Second,unknown,Cherbourg,n
1,female,4.0,1,1,16.7,Third,G,Southampton,n


In [6]:
LABEL_COLUMN = 'survived'
LABELS = [0, 1]

In [7]:
def get_dataset(file_path, **kwargs):
    dataset = tf.data.experimental.make_csv_dataset(
        file_path,
        batch_size=5, # 方便展示结果
        label_name=LABEL_COLUMN,
        na_value='?',
        num_epochs=1,
        ignore_errors=True,
        **kwargs
    )
    return dataset

raw_train_data = get_dataset(train_file_path)
raw_test_data = get_dataset(test_file_path)

Instructions for updating:
Use `tf.data.Dataset.interleave(map_func, cycle_length, block_length, num_parallel_calls=tf.data.experimental.AUTOTUNE)` instead. If sloppy execution is desired, use `tf.data.Options.experimental_determinstic`.


In [11]:
print(raw_test_data)
print()
print(raw_test_data)

<PrefetchDataset shapes: (OrderedDict([(sex, (None,)), (age, (None,)), (n_siblings_spouses, (None,)), (parch, (None,)), (fare, (None,)), (class, (None,)), (deck, (None,)), (embark_town, (None,)), (alone, (None,))]), (None,)), types: (OrderedDict([(sex, tf.string), (age, tf.float32), (n_siblings_spouses, tf.int32), (parch, tf.int32), (fare, tf.float32), (class, tf.string), (deck, tf.string), (embark_town, tf.string), (alone, tf.string)]), tf.int32)>

<PrefetchDataset shapes: (OrderedDict([(sex, (None,)), (age, (None,)), (n_siblings_spouses, (None,)), (parch, (None,)), (fare, (None,)), (class, (None,)), (deck, (None,)), (embark_town, (None,)), (alone, (None,))]), (None,)), types: (OrderedDict([(sex, tf.string), (age, tf.float32), (n_siblings_spouses, tf.int32), (parch, tf.int32), (fare, tf.float32), (class, tf.string), (deck, tf.string), (embark_town, tf.string), (alone, tf.string)]), tf.int32)>


In [12]:
def show_batch(dataset):
    for batch, label in dataset.take(1):
#         print(batch)
#         print(label)
        for key, value in batch.items():
            print('{:20s}: {}'.format(key, value.numpy()))

In [13]:
show_batch(raw_train_data)

sex                 : [b'male' b'female' b'male' b'male' b'male']
age                 : [28. 29. 16. 25. 16.]
n_siblings_spouses  : [0 1 0 0 0]
parch               : [0 1 0 0 0]
fare                : [ 7.775 10.462 26.     7.65   8.05 ]
class               : [b'Third' b'Third' b'Second' b'Third' b'Third']
deck                : [b'unknown' b'G' b'unknown' b'F' b'unknown']
embark_town         : [b'Southampton' b'Southampton' b'Southampton' b'Southampton'
 b'Southampton']
alone               : [b'y' b'n' b'y' b'y' b'y']


In [14]:
for feature, label in raw_train_data.take(1):
    print(feature)
    print(label)

OrderedDict([('sex', <tf.Tensor: id=195, shape=(5,), dtype=string, numpy=array([b'male', b'male', b'male', b'male', b'male'], dtype=object)>), ('age', <tf.Tensor: id=187, shape=(5,), dtype=float32, numpy=array([35. , 70.5, 28. , 40. , 20. ], dtype=float32)>), ('n_siblings_spouses', <tf.Tensor: id=193, shape=(5,), dtype=int32, numpy=array([0, 0, 0, 0, 0], dtype=int32)>), ('parch', <tf.Tensor: id=194, shape=(5,), dtype=int32, numpy=array([0, 0, 0, 0, 0], dtype=int32)>), ('fare', <tf.Tensor: id=192, shape=(5,), dtype=float32, numpy=array([ 7.896,  7.75 ,  7.796, 27.721,  9.846], dtype=float32)>), ('class', <tf.Tensor: id=189, shape=(5,), dtype=string, numpy=array([b'Third', b'Third', b'Third', b'First', b'Third'], dtype=object)>), ('deck', <tf.Tensor: id=190, shape=(5,), dtype=string, numpy=
array([b'unknown', b'unknown', b'unknown', b'unknown', b'unknown'],
      dtype=object)>), ('embark_town', <tf.Tensor: id=191, shape=(5,), dtype=string, numpy=
array([b'Cherbourg', b'Queenstown', b'So

In [15]:
SELECT_COLUMNS = ['survived', 'age', 'n_siblings_spouses', 'class', 'deck', 'alone']
temp_dataset = get_dataset(train_file_path, select_columns=SELECT_COLUMNS)
show_batch(temp_dataset)

age                 : [11. 30. 28. 27. 28.]
n_siblings_spouses  : [0 0 0 1 0]
class               : [b'Third' b'First' b'First' b'First' b'Second']
deck                : [b'unknown' b'B' b'C' b'E' b'unknown']
alone               : [b'y' b'y' b'y' b'n' b'y']


In [16]:
SELECT_COLUMNS = ['survived', 'age', 'n_siblings_spouses', 'parch', 'fare']
DEFAULTS = [0, 0.0, 0.0, 0.0, 0.0]
temp_dataset = get_dataset(train_file_path,
                          select_columns=SELECT_COLUMNS,
                          column_defaults=DEFAULTS)
show_batch(temp_dataset)

age                 : [31. 45. 28. 28. 20.]
n_siblings_spouses  : [1. 0. 0. 0. 0.]
parch               : [1. 0. 0. 0. 0.]
fare                : [20.525  7.75   7.733 15.5    8.05 ]


In [17]:
example_batch, labels_batch = next(iter(temp_dataset))

In [18]:
example_batch, labels_batch

(OrderedDict([('age',
               <tf.Tensor: id=348, shape=(5,), dtype=float32, numpy=array([22., 40., 19., 28., 16.], dtype=float32)>),
              ('n_siblings_spouses',
               <tf.Tensor: id=350, shape=(5,), dtype=float32, numpy=array([0., 0., 0., 0., 0.], dtype=float32)>),
              ('parch',
               <tf.Tensor: id=351, shape=(5,), dtype=float32, numpy=array([0., 0., 0., 0., 0.], dtype=float32)>),
              ('fare',
               <tf.Tensor: id=349, shape=(5,), dtype=float32, numpy=array([151.55 ,   7.896,   7.896,  26.   ,  26.   ], dtype=float32)>)]),
 <tf.Tensor: id=352, shape=(5,), dtype=int32, numpy=array([1, 0, 0, 0, 0], dtype=int32)>)

In [19]:
def pack(features, label):
    return tf.stack(list(features.values()), axis=-1), label

packed_dataset = temp_dataset.map(pack)


for features, labels in packed_dataset.take(1):
    print(features)
    print()
    print(labels)

tf.Tensor(
[[33.     0.     0.    12.275]
 [28.     0.     1.    33.   ]
 [21.     0.     0.    73.5  ]
 [ 7.     0.     2.    26.25 ]
 [28.     1.     0.    24.15 ]], shape=(5, 4), dtype=float32)

tf.Tensor([0 0 0 1 0], shape=(5,), dtype=int32)


In [20]:
for feature, label in raw_train_data.take(1):
    print(feature.values())
    print(feature.keys())
    print(feature)

odict_values([<tf.Tensor: id=388, shape=(5,), dtype=string, numpy=array([b'male', b'male', b'female', b'male', b'male'], dtype=object)>, <tf.Tensor: id=380, shape=(5,), dtype=float32, numpy=array([28., 40., 24., 28., 28.], dtype=float32)>, <tf.Tensor: id=386, shape=(5,), dtype=int32, numpy=array([0, 1, 0, 0, 0], dtype=int32)>, <tf.Tensor: id=387, shape=(5,), dtype=int32, numpy=array([0, 4, 0, 0, 0], dtype=int32)>, <tf.Tensor: id=385, shape=(5,), dtype=float32, numpy=array([47.1  , 27.9  , 49.504, 13.   ,  0.   ], dtype=float32)>, <tf.Tensor: id=382, shape=(5,), dtype=string, numpy=array([b'First', b'Third', b'First', b'Second', b'Second'], dtype=object)>, <tf.Tensor: id=383, shape=(5,), dtype=string, numpy=array([b'unknown', b'unknown', b'C', b'unknown', b'unknown'], dtype=object)>, <tf.Tensor: id=384, shape=(5,), dtype=string, numpy=
array([b'Southampton', b'Southampton', b'Cherbourg', b'Southampton',
       b'Southampton'], dtype=object)>, <tf.Tensor: id=381, shape=(5,), dtype=string

In [21]:
dataset_pked = raw_train_data.map(pack)
for feature, label in dataset_pked.take(1):
    print(feature)
    print()
    print(label)

TypeError: in converted code:

    <ipython-input-19-03e1bbc0deba>:2 pack  *
        return tf.stack(list(features.values()), axis=-1), label
    /home/kdd/.local/lib/python3.6/site-packages/tensorflow_core/python/util/dispatch.py:180 wrapper
        return target(*args, **kwargs)
    /home/kdd/.local/lib/python3.6/site-packages/tensorflow_core/python/ops/array_ops.py:1165 stack
        return gen_array_ops.pack(values, axis=axis, name=name)
    /home/kdd/.local/lib/python3.6/site-packages/tensorflow_core/python/ops/gen_array_ops.py:6304 pack
        "Pack", values=values, axis=axis, name=name)
    /home/kdd/.local/lib/python3.6/site-packages/tensorflow_core/python/framework/op_def_library.py:499 _apply_op_helper
        raise TypeError("%s that don't all match." % prefix)

    TypeError: Tensors in list passed to 'values' of 'Pack' Op have types [string, float32, int32, int32, float32, string, string, string, string] that don't all match.


In [22]:
STR_COLS = ['survived', 'sex', 'class', 'deck', 'embark_town', 'alone']
tmp_dataset = get_dataset(train_file_path, select_columns=STR_COLS)
show_batch(tmp_dataset)

sex                 : [b'male' b'female' b'female' b'female' b'female']
class               : [b'First' b'Third' b'Second' b'First' b'Second']
deck                : [b'E' b'unknown' b'E' b'C' b'unknown']
embark_town         : [b'Southampton' b'Cherbourg' b'Southampton' b'Cherbourg' b'Southampton']
alone               : [b'y' b'n' b'y' b'y' b'n']


In [23]:
class PackNumericFeature(object):
    def __init__(self, names):
        self.names = names
        
    def __call__(self, features, lables):
        numeric_features = [features.pop(name) for name in self.names]
        numeric_features = [tf.cast(feat, tf.float32) for feat in numeric_features]
        numeric_features = tf.stack(numeric_features, axis=-1)
        features['numeric'] = numeric_features
        return features, labels


In [24]:
NUMERIC_FEATURES = ['age','n_siblings_spouses','parch', 'fare']

packed_train_data = raw_train_data.map(PackNumericFeature(NUMERIC_FEATURES))
packed_test_data = raw_test_data.map(PackNumericFeature(NUMERIC_FEATURES))

In [25]:
show_batch(packed_train_data)

sex                 : [b'male' b'female' b'female' b'male' b'male']
class               : [b'Third' b'Third' b'First' b'First' b'Third']
deck                : [b'unknown' b'unknown' b'E' b'B' b'unknown']
embark_town         : [b'Cherbourg' b'Southampton' b'Southampton' b'Cherbourg' b'Southampton']
alone               : [b'y' b'n' b'n' b'n' b'y']
numeric             : [[28.     0.     0.     7.229]
 [18.     0.     1.     9.35 ]
 [28.     0.     1.    55.   ]
 [65.     0.     1.    61.979]
 [33.     0.     0.     8.654]]


In [26]:
example_batch, labels_batch = next(iter(packed_train_data))

In [27]:
example_batch, labels_batch

(OrderedDict([('sex',
               <tf.Tensor: id=553, shape=(5,), dtype=string, numpy=array([b'female', b'male', b'female', b'female', b'male'], dtype=object)>),
              ('class',
               <tf.Tensor: id=549, shape=(5,), dtype=string, numpy=array([b'Third', b'Third', b'Second', b'First', b'First'], dtype=object)>),
              ('deck',
               <tf.Tensor: id=550, shape=(5,), dtype=string, numpy=array([b'unknown', b'unknown', b'unknown', b'C', b'B'], dtype=object)>),
              ('embark_town',
               <tf.Tensor: id=551, shape=(5,), dtype=string, numpy=
               array([b'Queenstown', b'Southampton', b'Southampton', b'Southampton',
                      b'Cherbourg'], dtype=object)>),
              ('alone',
               <tf.Tensor: id=548, shape=(5,), dtype=string, numpy=array([b'y', b'y', b'n', b'y', b'y'], dtype=object)>),
              ('numeric',
               <tf.Tensor: id=552, shape=(5, 4), dtype=float32, numpy=
               array([[ 2

In [28]:
train_df = pd.read_csv(train_file_path)[NUMERIC_FEATURES]

In [29]:
train_df.head()

,age,n_siblings_spouses,parch,fare
0,22.0,1,0,7.2500
1,38.0,1,0,71.2833
2,26.0,0,0,7.9250
3,35.0,1,0,53.1000
4,28.0,0,0,8.4583


In [30]:
desc = train_df.describe()

In [31]:
desc

,age,n_siblings_spouses,parch,fare
count,627.000000,627.000000,627.000000,627.000000
mean,29.631308,0.545455,0.379585,34.385399
std,12.511818,1.151090,0.792999,54.597730
min,0.750000,0.000000,0.000000,0.000000
25%,23.000000,0.000000,0.000000,7.895800
50%,28.000000,0.000000,0.000000,15.045800
75%,35.000000,1.000000,0.000000,31.387500
max,80.000000,8.000000,5.000000,512.329200


In [32]:
MEAN = np.array(desc.T['mean'])
STD = np.array(desc.T['std'])

In [33]:
def normalize_numeric_data(data, mean, std):
    return (data - mean) / std

In [34]:
import functools
normalizer = functools.partial(normalize_numeric_data, mean=MEAN, std=STD)

In [35]:
numeric_column = tf.feature_column.numeric_column('numeric', normalizer_fn=normalizer, 
                                                 shape=[len(NUMERIC_FEATURES)])
numeric_columns = [numeric_column]
numeric_columns

[NumericColumn(key='numeric', shape=(4,), default_value=None, dtype=tf.float32, normalizer_fn=functools.partial(<function normalize_numeric_data at 0x7f83a0766598>, mean=array([29.631,  0.545,  0.38 , 34.385]), std=array([12.512,  1.151,  0.793, 54.598])))]

In [36]:
example_batch['numeric']

<tf.Tensor: id=552, shape=(5, 4), dtype=float32, numpy=
array([[ 28.   ,   0.   ,   0.   ,   7.75 ],
       [ 32.   ,   0.   ,   0.   ,   7.925],
       [ 19.   ,   1.   ,   0.   ,  26.   ],
       [ 40.   ,   0.   ,   0.   , 153.462],
       [ 46.   ,   0.   ,   0.   ,  79.2  ]], dtype=float32)>

In [37]:
numeric_layer = tf.keras.layers.DenseFeatures(numeric_columns)
numeric_layer(example_batch).numpy()

array([[-0.13 , -0.474, -0.479, -0.488],
       [ 0.189, -0.474, -0.479, -0.485],
       [-0.85 ,  0.395, -0.479, -0.154],
       [ 0.829, -0.474, -0.479,  2.181],
       [ 1.308, -0.474, -0.479,  0.821]], dtype=float32)

In [38]:
CATEGORIES = {
    'sex': ['male', 'female'],
    'class': ['First', 'Second', 'Third'],
    'deck': ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J'],
    'embark_town': ['Cherbourg', 'Southhampton', 'Queenstown'],
    'alone': ['y', 'n']
}

In [39]:
categorical_columns = []
for feature, vocab in CATEGORIES.items():
    cat_col = tf.feature_column.categorical_column_with_vocabulary_list(
        key=feature, vocabulary_list=vocab)
    categorical_columns.append(tf.feature_column.indicator_column(cat_col))

In [40]:
categorical_columns

[IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='sex', vocabulary_list=('male', 'female'), dtype=tf.string, default_value=-1, num_oov_buckets=0)),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='class', vocabulary_list=('First', 'Second', 'Third'), dtype=tf.string, default_value=-1, num_oov_buckets=0)),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='deck', vocabulary_list=('A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J'), dtype=tf.string, default_value=-1, num_oov_buckets=0)),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='embark_town', vocabulary_list=('Cherbourg', 'Southhampton', 'Queenstown'), dtype=tf.string, default_value=-1, num_oov_buckets=0)),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='alone', vocabulary_list=('y', 'n'), dtype=tf.string, default_value=-1, num_oov_buckets=0))]

In [41]:
categorical_layer = tf.keras.layers.DenseFeatures(categorical_columns)
print(categorical_layer(example_batch).numpy())  

Instructions for updating:
The old _FeatureColumn APIs are being deprecated. Please use the new FeatureColumn APIs instead.
Instructions for updating:
The old _FeatureColumn APIs are being deprecated. Please use the new FeatureColumn APIs instead.
[[1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 1.]
 [1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [1. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [1. 0. 1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 1. 0.]]


In [42]:
preprocessing_layer = tf.keras.layers.DenseFeatures(categorical_columns+numeric_columns)

In [43]:
preprocessing_layer(example_batch).numpy()

array([[ 1.   ,  0.   ,  0.   ,  0.   ,  1.   ,  0.   ,  0.   ,  0.   ,
         0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,
         0.   ,  1.   , -0.13 , -0.474, -0.479, -0.488,  0.   ,  1.   ],
       [ 1.   ,  0.   ,  0.   ,  0.   ,  1.   ,  0.   ,  0.   ,  0.   ,
         0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,
         0.   ,  0.   ,  0.189, -0.474, -0.479, -0.485,  1.   ,  0.   ],
       [ 0.   ,  1.   ,  0.   ,  1.   ,  0.   ,  0.   ,  0.   ,  0.   ,
         0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,
         0.   ,  0.   , -0.85 ,  0.395, -0.479, -0.154,  0.   ,  1.   ],
       [ 1.   ,  0.   ,  1.   ,  0.   ,  0.   ,  0.   ,  0.   ,  1.   ,
         0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,
         0.   ,  0.   ,  0.829, -0.474, -0.479,  2.181,  0.   ,  1.   ],
       [ 1.   ,  0.   ,  1.   ,  0.   ,  0.   ,  0.   ,  1.   ,  0.   ,
         0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0. 

In [44]:
model = tf.keras.Sequential([
    preprocessing_layer,
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [45]:
train_data = packed_train_data.shuffle(500)
test_data = packed_test_data
model.fit(train_data, epochs=20)

Epoch 1/20
     75/Unknown - 1s 10ms/step - loss: 0.5336 - accuracy: 0.7811

InvalidArgumentError:  Incompatible shapes: [2,1] vs. [5,1]
	 [[node BroadcastGradientArgs_3 (defined at /home/kdd/.local/lib/python3.6/site-packages/tensorflow_core/python/framework/ops.py:1751) ]] [Op:__inference_distributed_function_2338]

Function call stack:
distributed_function


In [ ]:
test_loss, test_accuracy = model.evaluate(test_data)

In [ ]:
predictions = model.predict(test_data)

# Show some results
for prediction, survived in zip(predictions[:10], list(test_data)[0][1][:10]):
  print("Predicted survival: {:.2%}".format(prediction[0]),
        " | Actual outcome: ",
        ("SURVIVED" if bool(survived) else "DIED"))